In [1]:
import random
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
import csv
import numpy as np
from tabulate import tabulate
from classification import classification
from rule_gen import rule_generation
from daehan_mlutil import utilities
from util import util

with open('../data/dataset.csv', 'r') as file:
    dataset = [list(filter(None, row)) for row in csv.reader(file)]

min_support = 0.1
transactions_df = util.convert_trans_to_df(dataset)
transactions_df

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,True,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,True,False,True,False,False,False,True,True,False,False,...,False,False,False,True,True,False,True,True,True,False
4611,True,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,True,False,False,False,False
4612,True,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
4613,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [2]:
random.seed(0)
transactions_0 = pd.DataFrame(
    transactions_df[transactions_df['0']].reset_index(drop=True))
transactions_1 = pd.DataFrame(
    transactions_df[transactions_df['1']].reset_index(drop=True))

In [3]:
indices = list(range(0, len(transactions_0)))
random.shuffle(indices)
test_set_0 = transactions_0.iloc[indices[:417], :].reset_index(drop=True)
training_set_0 = transactions_0.iloc[indices[417:], :].reset_index(drop=True)

indices = list(range(0, len(transactions_1)))
random.shuffle(indices)
test_set_1 = transactions_1.iloc[indices[:43], :].reset_index(drop=True)
training_set_1 = transactions_1.iloc[indices[43:], :].reset_index(drop=True)

training_set = pd.concat([training_set_0, training_set_1])
test_set = pd.concat([test_set_0, test_set_1])
training_set

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,True,True,False,False
1,True,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,True,False,True,False,False,False,True,True,False,False,...,False,True,False,False,False,False,True,True,False,False
3,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,False,True,True,False,False,False,True,True,False,False,...,False,False,False,False,False,True,False,False,False,False
389,False,True,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,True,True,False,False
390,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
391,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
test_set

,0,1,ACETAMINOPHEN,ALBUMIN HUMAN 25%,ALBUMIN HUMAN 5%,ALBUTEROL 0.083%,ALBUTEROL 90 MCG,ALBUTEROL CONCENTRATED 0.5%,ALLOPURINOL,ALPRAZOLAM,...,TACROLIMUS,TAMSULOSIN,THIAMINE,TRAMADOL,TRAZODONE,VANCOMYCIN,VANCOMYCIN 1 G/200 ML D5W,VANCOMYCIN 1.25 G/250 ML NS,VANCOMYCIN 1.5 G/500 ML NS,WARFARIN
0,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,True,False,False,False,True,True,False,False,...,False,False,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
39,False,True,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,False,True,False,False
40,False,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
41,False,True,True,True,False,False,False,True,False,False,...,False,False,True,False,False,False,True,False,False,False


In [5]:
import multiprocessing
import pandas as pd
from posneg_rule_gen.posneg_rule_generation import ponerg
from rule_gen import apriori_mlx
from util import util
from daehan_mlutil import utilities


def ponerg_parallel(item):
    return ponerg(item, classes, class_support_count_dict, transactions_df)

rules = []
transactions_df = training_set
X_df = pd.DataFrame(transactions_df.drop(['1', '0'], axis=1))
classes = [frozenset(['0']), frozenset(['1'])]
class_support_count_dict = util.get_support_count_dict_df(classes, transactions_df)

f1, previous_itemset_arr = apriori_mlx.apriori_of_size_1(X_df, min_support=min_support)
f1 = f1.tolist()
freq_itemsets = [f1]
for item in f1:
    rules.extend(ponerg(item, classes, class_support_count_dict, transactions_df))
np.expand_dims(np.array(np.array(f1)), axis=1)

array([[frozenset({'ACETAMINOPHEN'})],
       [frozenset({'ALBUTEROL 90 MCG'})],
       [frozenset({'ALBUTEROL CONCENTRATED 0.5%'})],
       [frozenset({'AMLODIPINE'})],
       [frozenset({'ASPIRIN'})],
       [frozenset({'ASPIRIN EC'})],
       [frozenset({"ASPIRIN, CHILDREN'S"})],
       [frozenset({'ATORVASTATIN'})],
       [frozenset({'BISACODYL'})],
       [frozenset({'CALCIUM GLUCONATE 2 G/50 ML NS'})],
       [frozenset({'CEFAZOLIN'})],
       [frozenset({'CEFAZOLIN 2 G/100 ML NS'})],
       [frozenset({'CEFEPIME'})],
       [frozenset({'CEFTRIAXONE'})],
       [frozenset({'CHLORHEXIDINE GLUCONATE 0.12%'})],
       [frozenset({'DEXTROSE 5%-NACL 0.9%'})],
       [frozenset({'DEXTROSE 50%-WATER'})],
       [frozenset({'DIAZEPAM'})],
       [frozenset({'DIPHENHYDRAMINE HCL'})],
       [frozenset({'DOCUSATE SODIUM'})],
       [frozenset({'ENOXAPARIN'})],
       [frozenset({'FAMOTIDINE'})],
       [frozenset({'FENTANYL 2000 MCG/100 ML NS'})],
       [frozenset({'FENTANYL CITRATE (PF)

In [6]:
import time

start_time = time.time()
k = 0
while freq_itemsets[k] is not None and len(freq_itemsets[k]) > 0:
    k_freq_itemsets, previous_itemset_arr = apriori_mlx.apriori_of_size_k(
        X_df, previous_itemset_arr, min_support=min_support, k=k + 2, low_memory=True)
    if not k_freq_itemsets.empty:
        k_freq_itemsets = k_freq_itemsets.tolist()
        # for item in k_freq_itemsets_x_c:
        #     rules.extend(ponerg(item, c, len(transactions_per_c), transactions_df))
        with multiprocessing.Pool() as pool:
            result = pool.map(ponerg_parallel, k_freq_itemsets)
        rules_to_extend = [x[0] for x in result if x != []]
        rules.extend(rules_to_extend)
        freq_itemsets.append(k_freq_itemsets)
    else:
        freq_itemsets.append(None)
    k += 1
time_sec = time.time() - start_time
time_min = time_sec / 60
print("\nProcessing time of %s(): %.2f seconds (%.2f minutes)."
      % ("rule_gen", time.time() - start_time, time_min))
np.expand_dims(np.array(np.array(rules)), axis=1)


Processing time of rule_gen(): 2.52 seconds (0.04 minutes).


array([[{'antecedent': frozenset({'ACETAMINOPHEN'}), 'consequent': '0', 'confidence': 0.2569277423402698}],
       [{'antecedent': frozenset({'ALBUTEROL 90 MCG'}), 'consequent': '1', 'confidence': 0.15285730471235215}],
       [{'antecedent': frozenset({'ALBUTEROL CONCENTRATED 0.5%'}), 'consequent': '1', 'confidence': 0.20172429358129462}],
       ...,
       [{'antecedent': frozenset({'MAGNESIUM SULF 2 G/50 ML NS', 'FENTANYL CITRATE (PF)', 'ACETAMINOPHEN', 'GLUCOSE', 'HEPARIN 5,000 UNIT/ML', 'DEXTROSE 50%-WATER', 'GLUCAGON', 'ONDANSETRON'}), 'consequent': '0', 'confidence': 0.6445846874763812}],
       [{'antecedent': frozenset({'MAGNESIUM SULF 2 G/50 ML NS', 'FENTANYL CITRATE (PF)', 'ACETAMINOPHEN', 'HEPARIN 5,000 UNIT/ML', 'DEXTROSE 50%-WATER', 'MIDAZOLAM', 'ONDANSETRON', 'OXYCODONE IR'}), 'consequent': '0', 'confidence': 0.6934907682472832}],
       [{'antecedent': frozenset({'MAGNESIUM SULF 2 G/50 ML NS', 'FENTANYL CITRATE (PF)', 'ACETAMINOPHEN', 'POTASSIUM CHLORIDE ER', 'HEPARIN 

In [12]:
import math

rules_0 = [rule for rule in rules if rule['consequent'] == '0']
rules_1 = [rule for rule in rules if rule['consequent'] == '1']
sorted_rules = sorted(rules, key=lambda d: d['confidence'], reverse=True)
freq_itemsets_count = sum([len(sublist) for sublist in freq_itemsets[:-1]])
sorted_rules = sorted_rules[:math.floor(freq_itemsets_count * 0.1) + 1]

print(freq_itemsets_count)
np.expand_dims(np.array(sorted_rules), axis=1)

23105


array([[{'antecedent': frozenset({'INSULIN REG 100 UNITS/100 ML', 'MAGNESIUM SULF 2 G/50 ML NS', 'POTASSIUM CHLORIDE ER', 'HEPARIN 5,000 UNIT/ML', 'OXYCODONE IR'}), 'consequent': '0', 'confidence': 0.9545248499573897}],
       [{'antecedent': frozenset({'INSULIN REG 100 UNITS/100 ML', 'MAGNESIUM SULF 2 G/50 ML NS', 'POTASSIUM CHLORIDE ER', 'HEPARIN 5,000 UNIT/ML', 'DEXTROSE 50%-WATER', 'OXYCODONE IR'}), 'consequent': '0', 'confidence': 0.9544165757906216}],
       [{'antecedent': frozenset({'MAGNESIUM SULF 2 G/50 ML NS', 'METOPROLOL TARTRATE', 'POTASSIUM CHLORIDE ER', 'OXYCODONE IR'}), 'consequent': '0', 'confidence': 0.9396689973699404}],
       ...,
       [{'antecedent': frozenset({'MAGNESIUM SULF 2 G/50 ML NS', 'DOCUSATE SODIUM', 'GLUCAGON'}), 'consequent': '0', 'confidence': 0.7380653981998404}],
       [{'antecedent': frozenset({'MAGNESIUM SULF 2 G/50 ML NS', 'GLUCAGON', 'DOCUSATE SODIUM', 'DEXTROSE 50%-WATER'}), 'consequent': '0', 'confidence': 0.7380653981998404}],
       [{'an

In [14]:
training_transactions_1 = training_set[training_set['1']].drop(['1', '0'], axis=1).apply(
    lambda row: frozenset(row.index[row]), axis=1).tolist()
scores_training = [classification.predict_proba(object_o, sorted_rules, True) for object_o in training_transactions_1]
scores_training = np.array(scores_training)
scores_training[scores_training == -1] = 0
scores_training

array([0.8280944 , 0.8346001 , 0.82300714, 0.8280944 , 0.82167637,
       0.84666445, 0.82300714, 0.78242314, 0.81950647, 0.8280944 ,
       0.82300714, 0.8280944 , 0.82167637, 0.82300714, 0.8280944 ,
       0.8280944 , 0.8280944 , 0.74172297, 0.8280944 , 0.        ,
       0.8280944 , 0.81950647, 0.82167637, 0.8176663 , 0.78242314,
       0.82300714, 0.82300714, 0.82300714, 0.8280944 , 0.88522159,
       0.8280944 , 0.8280944 , 0.8280944 , 0.81620739, 0.80584666,
       0.8280944 , 0.8280944 , 0.8280944 , 0.84007725, 0.82300714,
       0.82300714, 0.80132508, 0.78919195, 0.84380505, 0.8280944 ,
       0.82300714, 0.8280944 , 0.8280944 , 0.82300714, 0.86453982,
       0.78242314, 0.78242314, 0.82167637, 0.82677205, 0.86357455,
       0.8280944 , 0.8280944 , 0.8280944 , 0.93129771, 0.83040813,
       0.8280944 , 0.75860748, 0.8280944 , 0.8280944 , 0.8280944 ,
       0.8280944 , 0.82555774, 0.84045802, 0.78242314, 0.        ,
       0.8280944 , 0.82300714, 0.80800105, 0.82300714, 0.82809

In [9]:
mean = np.mean(scores_training)
mean

0.779568385998605

In [38]:
import importlib
importlib.reload(classification)
test_transactions = test_set.drop(['1', '0'], axis=1).apply(lambda row: frozenset(row.index[row]), axis=1).tolist()
scores = [classification.predict_proba_prune(object_o, sorted_rules) for object_o in test_transactions]
scores = np.array(scores)
scores

array([[0.        , 2.22994035],
       [2.49818181, 0.        ],
       [0.74028049, 1.55582473],
       [2.36567645, 0.        ],
       [0.        , 0.        ],
       [2.62858042, 0.        ],
       [2.84382225, 2.26677518],
       [0.        , 0.        ],
       [2.46254162, 0.        ],
       [2.81108614, 2.31964138],
       [2.84382225, 0.        ],
       [2.62708625, 0.        ],
       [0.        , 0.        ],
       [2.54204519, 2.29872845],
       [0.        , 2.29872845],
       [2.53262546, 1.55582473],
       [0.        , 2.45060099],
       [2.59537967, 0.        ],
       [2.5413905 , 1.55582473],
       [2.84861042, 2.31515689],
       [2.84382225, 2.4647329 ],
       [2.50236672, 0.        ],
       [2.40150973, 2.43789016],
       [2.84382225, 2.21913228],
       [2.73001601, 0.        ],
       [2.50236672, 0.        ],
       [2.73001601, 2.45458237],
       [0.        , 0.        ],
       [2.35964281, 2.29872845],
       [0.        , 0.        ],
       [2.

In [39]:
# y_pred = np.zeros(len(y_test), dtype=int)
# for i in range(scores_test.shape[0]):
#     if scores_test[i, 0] == 0 and scores_test[i, 1] == 0:
#         y_pred[i] = -1
#     elif scores_test[i, 0] >= scores_test[i, 1]:
#         y_pred[i] = 0
#     else:
#         y_pred[i] = 1
y_pred = np.where(scores[:, 0] >= scores[:, 1], 0, 1)
y_pred[(scores[:, 0] == 0) & (scores[:, 1] == 0)] = -1
# y_pred = (scores_test[:, 0] < scores_test[:, 1]).astype(np.uint8)
y_pred

array([ 1,  0,  1,  0, -1,  0,  0, -1,  0,  0,  0,  0, -1,  0,  1,  0,  1,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  0, -1,  0, -1,  0, -1,  0,  0,
        0,  0,  0, -1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0, -1,  0,  0,
        0,  1,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0,  1,  0,  0,  0,  0,
       -1,  0,  0,  0, -1, -1,  0,  0, -1, -1,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,
        0,  0,  0, -1,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,
       -1,  0,  0,  0, -1,  0,  1,  0,  0, -1,  0, -1,  0,  0,  0,  0,  0,
        0,  0, -1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, -1,  0,  0

In [40]:
not_classified = np.sum(y_pred == -1)
not_classified

50

In [41]:
y_pred[y_pred == -1] = 0
y_pred

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,

In [42]:
y_test = test_set.apply(lambda row: 0 if row['0'] else 1, axis=1).tolist()
y_test = np.array(y_test)
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [44]:
TP = np.sum(np.logical_and(y_pred == 1, y_test == 1))
TN = np.sum(np.logical_and(y_pred == 0, y_test == 0))
FP = np.sum(np.logical_and(y_pred == 1, y_test == 0))
FN = np.sum(np.logical_and(y_pred == 0, y_test == 1))
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2 * recall * precision / (recall + precision)
roc_auc = roc_auc_score(y_test, scores[:, 1])
accuracy = 100 * np.sum(y_test == y_pred) / len(y_test)
print(f"Pred as -1: {not_classified}")
print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"F1: {round(F1, 6)}")
print(f"roc auc: {roc_auc}")
print(f"Accuracy: {round(accuracy, 3)}%")
print(f"Total Rules: {len(sorted_rules)}")
print(f"Number of freq itemsets: {freq_itemsets_count}")
print(f"Rules with class 0: {len(rules_0)}")
print(f"Rules with class 1: {len(rules_1)}")
print(f"Avg rule conf: {round(sum(rule['confidence'] for rule in sorted_rules) / len(sorted_rules), 3)}")
print(f"Max rule conf: {round(sorted_rules[0]['confidence'], 3)}")
print(f"Min rule conf: {round(sorted_rules[-1]['confidence'], 3)}\n")
sorted_0 = [rule for rule in sorted_rules if rule['consequent'] == '0']
sorted_1 = [rule for rule in sorted_rules if rule['consequent'] == '1']
try:
    print(f"Avg conf for c0 rules: {round(sum(rule['confidence'] for rule in sorted_0) / len(sorted_0), 3)}")
    print(f"Max conf for c0 rules: {round(sorted_0[0]['confidence'], 3)}")
    print(f"Min conf for c0 rules: {round(sorted_0[-1]['confidence'], 3)}")
    print(f"Avg conf for c1 rules: {round(sum(rule['confidence'] for rule in sorted_1) / len(sorted_1), 3)}")
    print(f"Max conf for c1 rules: {round(sorted_1[0]['confidence'], 3)}")
    print(f"Min conf for c1 rules: {round(sorted_1[-1]['confidence'], 3)}")
except Exception:
    pass

Pred as -1: 50
Precision: 0.448
Recall: 0.605
F1: 0.514851
roc auc: 0.8460487424014277
Accuracy: 89.348%
Total Rules: 2311
Number of freq itemsets: 23105
Rules with class 0: 15225
Rules with class 1: 7880
Avg rule conf: 0.794
Max rule conf: 0.955
Min rule conf: 0.738

Avg conf for c0 rules: 0.796
Max conf for c0 rules: 0.955
Min conf for c0 rules: 0.738
Avg conf for c1 rules: 0.761
Max conf for c1 rules: 0.828
Min conf for c1 rules: 0.738
